In [1]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import context

from src.dataset import retrieve_dataloaders as loader
from src.model import LorentzNet
from src.train import train_loop, test_loop, forward_fn

/root/miniconda3/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/root/miniconda3/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/root/miniconda3/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/root/miniconda3/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
dataset, dataloaders = loader(32, 100000)

In [3]:
model = LorentzNet(n_scalar = 8, n_hidden = 72, n_class = 2,
                       dropout = 0.2, n_layers = 6,
                       c_weight = 0.001)

[WARNING] DEVICE(83158,ffff85e77020,python):2025-08-28-14:39:55.859.101 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:149] CheckVmmDriverVersion] Open file /etc/ascend_install.info failed.
[WARNING] DEVICE(83158,ffff85e77020,python):2025-08-28-14:39:55.859.176 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:177] CheckVmmDriverVersion] Open file /usr/local/Ascend/driver/version.info failed.


In [4]:
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=0.0002, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

In [5]:
grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
print('Train')
file_name = 'LorentzNet_training'
for t in range(3):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, dataloaders['train'], loss_fn, grad_fn, optimizer, t, file_name)
    print()
    test_loop(model, dataloaders['val'], loss_fn)


print('Test')
test_loop(model, dataloaders['test'], loss_fn)
print("Done!")

Train
Epoch 1
-------------------------------
 [**************************************************]2500/2500 - used 1150.0s / left 0.0s / loss 0.4910376953 / acc 78.0138   
.Valid: 
 Accuracy: 79.2%, Avg loss: 0.469650 

Epoch 2
-------------------------------
 [**************************************************]2500/2500 - used 1069.9s / left 0.0s / loss 0.4683453613 / acc 79.4150  
Valid: 
 Accuracy: 79.0%, Avg loss: 0.454286 

Epoch 3
-------------------------------
 [**************************************************]2500/2500 - used 1123.1s / left 0.0s / loss 0.4595571777 / acc 79.8762   
Valid: 
 Accuracy: 80.0%, Avg loss: 0.445712 

Test
Valid: 
 Accuracy: 80.1%, Avg loss: 0.442475 

Done!
